In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import pairwise_distances, accuracy_score, recall_score, roc_auc_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV

import graphics_utility as gru

from pathlib import Path
import os
import pickle

import dtw_metrics_tslearn as dtm

In [2]:
feature16_path = Path('../data/features_16')
feature32_path = Path('../data/features_32')

In [3]:
f16_path_dictionary = {}

for sub_path in feature16_path.iterdir():
    if False==sub_path.is_dir():
        continue
    f16_path_dictionary[sub_path.name] = {'mfcc': [], 'cq': [], 'wp': []}
    for item in sub_path.iterdir():
        if False== item.is_file() or item.suffix!='.npy':
            continue
        if '_mfcc' in item.stem:
            f16_path_dictionary[sub_path.name]['mfcc'].append(item.absolute())
        elif '_cq' in item.stem:
            f16_path_dictionary[sub_path.name]['cq'].append(item.absolute())
        elif '_wp' in item.stem:
            f16_path_dictionary[sub_path.name]['wp'].append(item.absolute())
        else:
            continue
    f16_path_dictionary[sub_path.name]['mfcc'].sort()
    f16_path_dictionary[sub_path.name]['cq'].sort()
    f16_path_dictionary[sub_path.name]['wp'].sort()

In [4]:
def get_multiclass_dataset(path_dictionary, feature, key_words=None):
    '''
    Parameters:
    path_dictionary : dict of paths to feature folders
    feature (str)   : 'wp' - wavelet packets envelopes, 'mfcc', 'cq' - constant Q spectrum
    '''
    if feature not in ['wp', 'mfcc', 'cq']:
        print("feature name can be either 'wp' or 'mfcc' or 'cq'")
        return None
    
    raw_X = []
    max_size = 0
    y = []
    label_dict = {}
    if key_words is None:
        key_words = list(path_dictionary.keys())
    for word_idx, word in enumerate(path_dictionary.keys()):
        if word in key_words:
            label_dict[word] = key_words.index(word)+1
        else:
            label_dict[word] = 0
            
        for path in path_dictionary[word][feature]:
            sample = np.load(path).flatten()
            if sample.size > max_size:
                max_size = sample.size
            raw_X.append(sample)
            if word in key_words:
                y.append(key_words.index(word)+1)
            else:
                y.append(0)
            
    X = []
    for sample in raw_X:
        padded_sample = np.zeros(max_size)
        padded_sample[0:sample.size]=sample[0:sample.size]
        X.append(padded_sample)
    return np.array(X), np.array(y), label_dict

In [5]:
X, y, label_dict = get_multiclass_dataset(f16_path_dictionary, 'mfcc', key_words=None)

In [6]:
X[0].size

944

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

#### **Для сравнения с KNN протестируем качество распознавания команд**
#### **нейронной сетью простейшей архитектуры.**

In [8]:
mlp = MLPClassifier(
    hidden_layer_sizes=(2000,1000),
    max_iter=500, alpha=1e-4,
    solver='sgd', verbose=0, random_state=1812,
    learning_rate_init=.001
)

In [9]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(2000, 1000), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1812, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)

In [10]:
y_pred = mlp.predict(X_test)
y_pred

array([ 3,  3,  8,  3,  5, 10,  2,  7,  5,  6,  5,  1,  9, 10,  6,  6,  2,
        7,  4,  2,  6,  8,  2, 10,  1,  7,  4,  3,  6,  2,  4,  6,  8,  5,
        3,  4,  7,  7, 10,  9,  6, 10, 10,  3,  1,  7,  6,  7,  6,  9])

In [11]:
y_test

array([ 3,  5,  7,  5,  7,  2, 10,  7,  5,  6,  4,  1,  9, 10,  9,  2,  1,
        7,  4,  1,  6,  8,  2,  2,  4,  1,  4,  3,  6,  2,  5,  5,  8,  3,
        9,  4,  7,  9, 10,  9,  6, 10, 10,  3,  1,  1,  2,  2,  6,  9])

In [12]:
f1_score(y_test, y_pred, average='macro')

0.5674891774891775

In [13]:
accuracy_score(y_test, y_pred)

0.56

#### Вывод: нейронная сеть даже такой простой архитектуры требует в десятки и сотни раз
#### вычислительных операций, чем KNN, при этом качество распознавания несопоставимо хуже.